In [3]:
# 循环神经网络常常由于梯度衰减的缘故造成
# 它无法在实际应用中博捉到时间序列中较大的依赖关系
# gated recurrent NN 通过可以学习的门来控制信息的流动
# Gated recurrent unit是一种常用的门控循环神经网络
# LSTM是另一种

# 门控循环单元 引用了重置门 reset gate 和更新门 update gate
# 从而修改了循环神经网络中隐藏状态的计算方式
# 重置门和更新门 （LSTM 输入输出和遗忘门）
# 重置门和更新门的输入均为当前时间步的输入Xt和上一时间步的隐藏状态Ht-1 ,输出由激活函数为Sigmoid的全连接层得到
# 重置门和更新门的输出的值域是【0,1】

# 候选隐藏状态
# 重置门控制了上一步的隐藏状态如何流入当前时间步的候选隐藏状态，可以用来丢弃
# 与预测无关的历史信息

# 隐藏状态
# 重置门有助于捕捉时间序列短期的依赖关系
# 更新门有助于捕捉时间序列里长期的依赖关系（可以有效缓解梯度衰减的问题）

# 读取数据集
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F

import sys 
sys.path.append('..')
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()


# 从零开始实现GRU
# 首先介绍如何从零开始实现门控循环单元
# 初始化模型参数

num_inputs,num_hiddens,num_outputs = vocab_size,256,vocab_size
print('will use',device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size = shape),device = device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    
    def _three():
        return (_one((num_inputs,num_hiddens)),
               _one((num_hiddens,num_hiddens)),
               torch.nn.Parameter(torch.zeros(num_hiddens,device=device,dtype=torch.float32),requires_grad=True))
    
    W_xz,W_hz,b_z = _three()# 更新门参数
    W_xr,W_hr,b_r = _three()# 重置门参数
    W_xh,W_hh,b_h = _three()# 候选隐藏状态参数
    
    # 输出参数
    W_hq = _one((num_hiddens,num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs),device=device,dtype = torch.float32,requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

def init_gru_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device),) # ?? why

def gru(inputs,state,params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X,W_xz) + torch.matmul(H,W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X,W_xr) + torch.matmul(H,W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X,W_xh) + R*torch.matmul(H,W_hh) + b_h)
        H = Z*H + (1-Z)*H_tilda
        Y = torch.matmul(H,W_hq) + b_q
        outputs.append(Y)
        
    return outputs,(H,)

# 训练模型并创作歌词

num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']


d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)
    

will use cpu


TypeError: __new__() got an unexpected keyword argument 'device'

In [ ]:
# 简洁实现
# 在pytorch中我们直接调用nn模块中的GRU类即可
lr = 1e-2
gru_layer = nn.GRU(input_size = vocab_size,hidden_size = num_hiddens)